In [ ]:
%pylab inline

import warnings
warnings.filterwarnings("ignore")

from SCCAF import *
import os
os.chdir("./RETINA/")


In [ ]:
def label_projection (adTr, adLn, key = 'org_cluster', mode = 'X', 
                      n_HVG = 100, n=50, frac=0.8, 
                      Ln_species = 'Patch', Tr_species = 'TenX',
                      plot_tsne = False):
# SCCAF projectin
#input: adTr: training set; adLn, testing set;key, label for learning; mode, X or PCA or harmony PCA
#output: dataframe of probablity of each cell, each label
    
    adTr_sub = data_subset(adTr)
    TrLn = sele_dataset(adTr_sub, adLn, n_HVG = n_HVG,Ln_species = Ln_species, Tr_species = Tr_species)
    print('get dataset')
    
    if mode == 'X':
        Tr_sele = TrLn[TrLn.obs['source']==Tr_species,:]
        Ln_sele = TrLn[TrLn.obs['source']==Ln_species,:]
        Tr_matrix = Tr_sele.X
        Ln_matrix = Ln_sele.X
        print('split dataset')
        
    elif mode == 'X_pca':
          

        TrLn = run_PCA_Harmony(TrLn, run_Harmony = False, theta = 5,rep = 3)
        Tr_sele = TrLn[TrLn.obs['source']==Tr_species,:]
        Ln_sele = TrLn[TrLn.obs['source']==Ln_species,:]
        
        if scipy.sparse.issparse(adTr.obsm['X_pca']): 
            Tr_matrix  = Tr_sele.obsm['X_pca'].todense()
        else:
            Tr_matrix  = Tr_sele.obsm['X_pca']
            
        if scipy.sparse.issparse(adLn.obsm['X_pca']): 
            Ln_matrix  = Ln_sele.obsm['X_pca'].todense()
        else:
            Ln_matrix  = Ln_sele.obsm['X_pca']
            
        
    elif mode == 'X_harmony':    
        adTr_sub = data_subset(adTr)
        TrLn = sele_dataset(adTr_sub, adLn, n_HVG = n_HVG,Ln_species = Ln_species, Tr_species = Tr_species)  
        
        TrLn = run_PCA_Harmony(TrLn, run_Harmony = True, theta = 5,rep = 4)
        Tr_sele = TrLn[TrLn.obs['source']==Tr_species,:]
        Ln_sele = TrLn[TrLn.obs['source']==Ln_species,:]
        
        if scipy.sparse.issparse(Tr_sele.obsm['X_harmony']): 
            Tr_matrix  = Tr_sele.obsm['X_harmony'].todense()
        else:
            Tr_matrix  = Tr_sele.obsm['X_harmony']
            
        if scipy.sparse.issparse(Ln_sele.obsm['X_harmony']): 
            Ln_matrix  = Ln_sele.obsm['X_harmony'].todense()
        else:
            Ln_matrix  = Ln_sele.obsm['X_harmony']
        
    print('split learning')    
    X_train, X_test, y_train, y_test = train_test_split_per_type(Tr_matrix, Tr_sele.obs[key], n=n, frac=frac)
    
    print('regression')
    clf = LogisticRegression(random_state=1, penalty='l1', C=0.5, solver = 'saga')
    
    clf.fit(X_train, y_train)

    if plot_tsne:
        sc.pl.umap(adTr, color = [key])

    Tr_sele.obs[key+'_predict'] = clf.predict(Tr_matrix)
    if plot_tsne:
        sc.pl.tsne(adTr, color = [key,key+'_predict'])
    print('predicting')
    adLn.obs[key + '_predict'] = clf.predict(Ln_matrix)
    #sc.pl.tsne(adLn, color = [key,key+'_predict'])
    df_prob = pd.DataFrame(clf.predict_proba(Ln_matrix),index = adLn.obs_names, columns = clf.classes_)
    
    return df_prob

In [ ]:
#load 10X
ad10x = sc.read('data/onc_atlas.h5')


In [ ]:
#check_results
sc.pl.umap(ad10x, color = 'org_cluster')

In [ ]:
#data pre_processing
figsize(4,4)
ad10x.X = ad10x.layers['counts']
sc.pp.normalize_per_cell(ad10x, counts_per_cell_after=1e4)
sc.pp.log1p(ad10x)
ad10x.layers['raw'] = ad10x.X
ad10x.X

In [ ]:
#load patch-seq data
adpatch = sc.read('data/retina_patch_all_qc.h5') 

In [ ]:
#patch-seq data preprocessing
figsize(4,4)
adpatch.X = adpatch.layers['counts']
sc.pp.normalize_per_cell(adpatch, counts_per_cell_after=1e5)
sc.pp.log1p(adpatch)
adpatch.layers['raw'] = adpatch.X
adpatch.X

In [ ]:
#check common genes
common_genes = list((set(ad10x.var_names.tolist()))&(set(adpatch.var_names.tolist())))
len(common_genes)

In [ ]:
#use only the genes shared by the two datasets
ad10x = ad10x[:,common_genes]
ad10x
adpatch = adpatch[:,common_genes]
adpatch

In [ ]:
#Looping of SCCAF using different parameters

n_gene_list  = [2000,4000,6000,8000,10000]
mode_list = ['X','X_pca','X_harmony']
tenx_cell_list = [25, 50, 75, 100,200,1000]
N_rep_list =[1]
penelty_list = ['l1','l2']
C_value_list = [0.3,0.5,0.8]
solver_mode_list = ['saga']


for n_gene in n_gene_list:
    for mode_use in mode_list:
        for tenx_cell in tenx_cell_list:
            for N_rep in N_rep_list:
                for penelty in penelty_list:
                    for C_value in C_value_list:
                        for solver_mode in solver_mode_list:
                            param = f'HVG_{n_gene}_{mode_use}_Tenx_{tenx_cell}_rep_{N_rep}_pen_{penelty}_C_{C_value}_solver_{solver_mode}'
                            print(param)
                            
                            #skip calculated ones
                            finished = pd.read_csv('data/score_summary.csv')
                            if param in list(finished['Param']): 
                                    print('finished')
                                    continue
                            
                            #load data
                            ad10X, adpatch = load_h5(path_tenx = 'data/onc_atlas.h5', 
                                            path_patch = 'data/retina_patch_all_qc.h5' )
                            
                            #start of the function
                            df_prob_rep = pd.Series()
                            for rep in range(0, N_rep):
                                print('repeat ' + str(rep))
                                
                                df_prob = pd.DataFrame()
                                df_prob = label_projection(adTr = ad10x, adLn = adpatch,key = 'org_cluster',  Tr_species = 'TenX',Ln_species = 'Patch', 
                                                          mode = mode_use, n_HVG = n_gene, n=tenx_cell, frac=0.8,
                                                           panelty = penelty, C_value = C_value, solver_mode = solver_mode,)
                                df_prob_rep['rep_'+str(rep)] = df_prob
                            
                            #calucate score and write to csv
                            score_sum, pre_results = find_save_score(df_prob_rep, manual_label, param)

# summary_all parameters

In [ ]:

n_gene_list  = [2000,4000,6000,8000,10000]
mode_list = ['X','X_pca','X_harmony']
tenx_cell_list = [25, 50, 100,200,1000]
N_rep_list =[1]
penelty_list = ['l1','l2']
C_value_list = [0.3,0.5,0.8]
solver_mode_list = ['saga']


para_list = []
for n_gene in n_gene_list:
    for mode_use in mode_list:
        for tenx_cell in tenx_cell_list:
            for N_rep in N_rep_list:
                for penelty in penelty_list:
                    for C_value in C_value_list:
                        for solver_mode in solver_mode_list:
                            param = f'HVG_{n_gene}_{mode_use}_Tenx_{tenx_cell}_rep_{N_rep}_pen_{penelty}_C_{C_value}_solver_{solver_mode}'
                            print(param)
                            para_list.append(param)
len(para_list)

In [ ]:
len(para_list)

In [ ]:
df_pred_all = pd.DataFrame(index = list(adpatch.obs_names))

for para in para_list:
    #print(para)
    import pickle
    with open(f'data/predict_results_all/{para}.pckl', 'rb') as f:
        x = pickle.load(f)
    df_pred_all[para] = x['rep_0'].idxmax(axis = 1)
    
df_pred_all.to_csv('results/df_predict_all_param.csv')  

In [ ]:
label_summary_all = pd.DataFrame(index = df_pred_tops.index)

label_summary_all['best_fit'] = np.nan
label_summary_all['best_fit_score'] = np.nan

label_summary_all['2nd_fit'] = np.nan
label_summary_all['2nd_fit_score'] = np.nan


all_cells = df_pred_all.index.tolist()
for cell in all_cells:
    counts_info = df_pred_all.loc[cell,:].value_counts()
    label_summary_all.loc[cell,'best_fit'] = counts_info.index.tolist()[0]
    label_summary_all.loc[cell,'best_fit_score'] = counts_info[0]/sum(counts_info)
    label_summary_all.loc[cell,'2nd_fit'] = counts_info.index.tolist()[(int(len(counts_info)>1))]
    label_summary_all.loc[cell,'2nd_fit_score'] = counts_info[(int(len(counts_info)>1))]/sum(counts_info)
label_summary_all

In [ ]:
hist(label_summary_all['best_fit_score'])

In [ ]:
label_summary_all['best_fit_score'].mean()

In [ ]:
label_summary_all['best_fit_score'].std()

In [ ]:
label_summary_all

In [ ]:
label_summary_all.to_csv('data/SCCAF_projection_retina_all.csv')